In [ ]:
import json
import os
import os.path
import numpy as np
import pandas as pd
import random
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
# Resolved an issue where images are too large to be processed by PIL
import torch
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset, DataLoader
import  torchvision
from torchvision import datasets, transforms
torch.cuda.is_available()
device = torch.device("cuda:0")
device
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
!nvidia-smi

In [4]:
def read_split_data(root: str, tra_rate: float = 0.8, val_rate: float = 0.1, test_rate: float = 0.1):
    random.seed(42)  
    assert os.path.exists(
        root), "dataset root: {} does not exist.".format(root)

    object_class = [cla for cla in sorted(os.listdir(root)) if os.path.isdir(os.path.join(root, cla))]
    num_classes = len(object_class)

    class_indices = dict((k, v) for v, k in enumerate(object_class))
    json_str = json.dumps(dict((val, key) for key, val in class_indices.items()), indent=4)
    with open('class_indices.json', 'w') as json_file:
        json_file.write(json_str)

    train_images_path = []  
    train_images_label = [] 
    val_images_path = []  
    val_images_label = [] 
    test_images_path = []  
    test_images_label = []  

    every_class_num = []  
    supported = [".jpg", ".JPG", ".png", ".PNG"]  
   
    for cla in object_class:
        cla_path = os.path.join(root, cla)
        images = [os.path.join(root, cla, i) for i in sorted(os.listdir(cla_path))
                  if os.path.splitext(i)[-1] in supported]
        image_class = class_indices[cla]
        total = len(images)
        every_class_num.append(total)
        random.shuffle(images)
        train_images = images[0:int(tra_rate*total)]  
        val_images = images[int(tra_rate*total):int((tra_rate+val_rate)*total)] 
        test_images = images[int((tra_rate+val_rate)*total):]

        for img_path in images:
            if img_path in train_images:
                train_images_path.append(img_path)
                train_images_label.append(image_class)
            if img_path in val_images:
                val_images_path.append(img_path)
                val_images_label.append(image_class)
            if img_path in test_images:
                test_images_path.append(img_path)
                test_images_label.append(image_class)
        print(cla, ':', 'count:', total, ' train:', len(train_images),
              'val:', len(val_images), 'test:', len(test_images))

    print(
        f"There are {len(object_class)} class and {sum(every_class_num)} images were found in the dataset.")
    print("{} images for train.".format(len(train_images_path)))
    print("{} images for validation.".format(len(val_images_path)))
    print("{} images for test.".format(len(test_images_path)))

    plot_image = True
    save_image = True
   
    return train_images_path, val_images_path, test_images_path ,num_classes

In [ ]:
path1 = r'./Datasets/AiRound/aerial/'
path2 = r'./Datasets/New_AiRound/aerial/'
train_images_path1, train_images_label1,val_images_path, val_images_label,test_images_path, test_images_label=read_split_data(path1,0.7,0.1,0.2)
train_images_path2, train_images_label2,_,_,_,_=read_split_data(path2,1,0,0)
dele_path = val_images_path+test_images_path
dele_label = val_images_label+test_images_label

In [ ]:
def select_path(dele_path , train_images_path):
    dele = []
    for i in range(0 , len(dele_path)):
        dele.append(dele_path[i].split("/")[-2]+"/"+ dele_path[i].split("/")[-1].split('.')[0])
    whole = []
    for k in range(0 , len(train_images_path2)):
        whole.append(train_images_path[k].split('/')[-2]+"/"+train_images_path[k].split('/')[-1])
    
    conindex = []
    unconindex = []
    con =[]
    uncon = []
    for t in range(0,len(whole)):
        if whole[t].split('.')[0] in dele:
            con.append(t)
            conindex.append(whole[t])
        else:
            uncon.append(t)
            unconindex.append(whole[t])
    return con , conindex , uncon , unconindex

def trans(label,con,uncon):
    scon =[]
    suncon = []
    for i in con:
        scon.append(label[i])
    for j in uncon:
        suncon.append(label[j])
    return scon,suncon



con , conindex , uncon , unconindex = select_path(dele_path , train_images_path2) 
new_con,new_uncon = trans(train_images_path2,con,uncon)
new_conlabel,new_unconlabel = trans(train_images_label2,con,uncon)
#==========================================================#
train_dataset_path = new_uncon
train_dataset_label = new_unconlabel
clean_val_dataset_path = val_images_path
clean_val_dataset_label = val_images_label
clean_test_dataset_path = test_images_path
clean_test_dataset_label = test_images_label

In [ ]:
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
#     transforms.RandomResizedCrop(224,scale=(0.7,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.6959, 0.6537, 0.6371), (0.3113, 0.3192, 0.3214)),
])
transform_test = transforms.Compose([
    
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
#     transforms.RandomResizedCrop(224,scale=(0.7,1.0)),
    transforms.ToTensor(),
    transforms.Normalize((0.6959, 0.6537, 0.6371), (0.3113, 0.3192, 0.3214)),

])
class Add(Dataset):
    def __init__(self, Dataset,Labels,transform = None,target_transform = None):
        self.List = Dataset
        self.Labels = Labels
        self.length = len(self.List)
        if transform is None:
            self.transform = transforms.ToTensor()
        else:
            self.transform = transform
        self.target_transform = target_transform
    def __getitem__(self, index):
        img = Image.open(self.List[index]).convert("RGB")
        label = self.Labels[index]
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return img,label,index
    def __len__(self):
        return self.length
    def getData(self):
        return self.List, self.Labels

In [ ]:
train_images = Add(train_dataset_path,train_dataset_label,transform = transform_train)
val_images = Add(clean_val_dataset_path,clean_val_dataset_label,transform = transform_train)
test_images = Add(clean_test_dataset_path,clean_test_dataset_label,transform = transform_test)
train_loader = DataLoader(dataset=train_images, batch_size=16, num_workers=8, pin_memory=True, shuffle=True, drop_last=True)
val_loader = DataLoader(dataset=val_images, batch_size=8, num_workers=8, pin_memory=True, shuffle=False, drop_last=False)
test_loader = DataLoader(dataset=test_images, batch_size=8, num_workers=8, pin_memory=True, shuffle=False, drop_last=False)